In [18]:
import os
import fitz
import re
import json
from PyPDF2 import PdfReader

In [19]:
def extract_text_from_pdf(pdf_path, skip_pages=0):
    reader = PdfReader(pdf_path)
    text = ""
    for i, page in enumerate(reader.pages):
        if i < skip_pages:
            continue
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text

def parse_document_with_roman_chapters(text, doc_title):
    result = {
        "документ": doc_title,
        "главы": []
    }

    # Регулярка: римская цифра с точкой и названием
    chapter_pattern = re.compile(r"\b([IVXLCDM]+)\.\s+([^\n]+)")

    matches = list(chapter_pattern.finditer(text))
    for i, match in enumerate(matches):
        chapter_num = match.group(1) + "."
        chapter_title = match.group(2).strip()
        start = match.end()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)
        chapter_text = text[start:end]

        # Поиск пунктов: 1., 2., 3. и т.д.
        пункт_matches = list(re.finditer(r"\n(\d+)\.\s+(.*?)(?=\n\d+\.|\Z)", chapter_text, re.DOTALL))
        пункты = [
            {
                "номер": m.group(1) + ".",
                "текст": re.sub(r"\s+", " ", m.group(2)).strip()
            }
            for m in пункт_matches
        ]

        result["главы"].append({
            "глава": chapter_num,
            "заголовок": chapter_title,
            "пункты": пункты
        })

    return result


In [20]:

pdf_path = "C:\\Users\\My Computer\\Desktop\\Work\\Learn\\OPK_Project\\documents_pdf\\05_12_2013_N_457.pdf"
outp_path = "C:\\Users\\My Computer\\Desktop\\Work\\Learn\\OPK_Project\\documents_json\\05_12_2013_N_457.json"
doc_title = "АДМИНИСТРАТИВНЫЙ РЕГЛАМЕНТ ФЕДЕРАЛЬНОГО АГЕНТСТВА ВОЗДУШНОГО ТРАНСПОРТА ПРЕДОСТАВЛЕНИЯ ГОСУДАРСТВЕННОЙ УСЛУГИ ПО ГОСУДАРСТВЕННОЙ РЕГИСТРАЦИИ ГРАЖДАНСКИХ ВОЗДУШНЫХ СУДОВ И ВЕДЕНИЮ ГОСУДАРСТВЕННОГО РЕЕСТРА ГРАЖДАНСКИХ ВОЗДУШНЫХ СУДОВ РОССИЙСКОЙ ФЕДЕРАЦИИ Утвержден приказом Минтранса России от 05.12.2013 N 457 "

raw_text = extract_text_from_pdf(pdf_path, skip_pages=0)
parsed_json = parse_document_with_roman_chapters(raw_text, doc_title)

with open(outp_path, "w", encoding="utf-8") as f:
    json.dump(parsed_json, f, ensure_ascii=False, indent=2)